In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4


cpu


In [8]:
with open('Gita.txt','r', encoding='utf-8') as f:
    text = f.read()
    
chars = sorted(set(text))
print(chars)
vocublary_size = len(chars)
print(vocublary_size)



['\n', ' ', '!', '"', '#', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
79


In [9]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}

encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([ 0,  0, 28, 57, 56, 61, 55, 53, 72, 61, 67, 66,  0,  0, 44, 39,  1, 33,
        38, 28, 33, 25,  0,  0, 43, 67,  1, 60, 53, 74, 57,  1, 33,  1, 70, 57,
        53, 56,  1, 72, 60, 61, 71,  1, 75, 67, 66, 56, 57, 70, 58, 73, 64,  1,
        53, 66, 56,  1, 71, 68, 61, 70, 61, 72, 10, 72, 60, 70, 61, 64, 64, 61,
        66, 59,  1, 71, 68, 57, 57, 55, 60,  9,  0, 26, 77,  1, 35, 70, 61, 71,
        60, 66, 53,  1, 53, 66, 56,  1, 40, 70])


In [10]:
n = int(0.8*len(data))

train_data = data[:n]
val_data = data[n:]
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 9,  0,  1,  1, 44, 60, 67, 73],
        [10, 72, 60, 67, 73,  1, 54, 57],
        [75, 61, 72, 60, 61, 66,  0,  0],
        [57, 65, 54, 64, 53, 66, 55, 57]])
targets:
tensor([[ 0,  1,  1, 44, 60, 67, 73,  1],
        [72, 60, 67, 73,  1, 54, 57, 71],
        [61, 72, 60, 61, 66,  0,  0,  1],
        [65, 54, 64, 53, 66, 55, 57, 71]])


In [11]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([0]) target is tensor(0)
when input is tensor([0, 0]) target is tensor(28)
when input is tensor([ 0,  0, 28]) target is tensor(57)
when input is tensor([ 0,  0, 28, 57]) target is tensor(56)
when input is tensor([ 0,  0, 28, 57, 56]) target is tensor(61)
when input is tensor([ 0,  0, 28, 57, 56, 61]) target is tensor(55)
when input is tensor([ 0,  0, 28, 57, 56, 61, 55]) target is tensor(53)
when input is tensor([ 0,  0, 28, 57, 56, 61, 55, 53]) target is tensor(72)


In [12]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocublary_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocublary_size, vocublary_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :] 
            probs = F.softmax(logits, dim=-1) 
            index_next = torch.multinomial(probs, num_samples=1) 
            index = torch.cat((index, index_next), dim=1) 
        return index

model = BigramLanguageModel(vocublary_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


(-ekRe!7UPBV5t`!Kcg 
`D'0d,NT
f5F]sp.zClcJ"Yv xj0v wi.ko.IY63HpJb?f7lhUX-8PiXG"D5tbYr1`
,UimC-0qkIA-&iK[e9X&n(Lfx`]&pX.3rth2NvtUrBE[
n44K R]b ?mc?fGU]vS`X
I6v'&)08TJYr!63obB)QVS]!tQ)[4N'6d(caI]&1(Wge1D&CtP6mm8J&xi1`KcrnvDnI)v7!qV"rL("FRsaIFUQffwg3ahkcod"L#fawA)VWjWejb0i6YvOQVff'"YL2.)1iO3DQKR`zaI;'01zjLRu- Iok-cEI;41ozGkM!1q08j;I8uct,!mQz?)"Ov m`sWIP03nm)i9'fB)g`eA?SVCID;'P;`QKcpk"5bn8dl5zk1`lkCE?Xkctfu3(WIiSi[qQXXg9qlzDzoF-a#fS'`7H?'x[6Nm)88KzI7&RznkRwA-56m-;WyFPtb:xgHzP
nEX-5"WYLB9[;yn]2I]UJtb


In [13]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

4.963321685791016
